In [29]:
import pandas as pd

from langdetect import detect

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Byte Pair Encoding(BPE) -- Subword tokenization

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace


# To Download the resources
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ranjith/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("multilingual_twitter_dataset.csv")
df.head()

,username,location,gender,age,tweet
0,user_1,India,Female,43,Can't believe this happened... lol :D #fail 😅
1,user_2,France,Male,26,信じられない… 😭💔
2,user_3,Germany,Other,30,Das ist fantastisch 😍💯! #Liebe
3,user_4,Germany,Female,38,Can't believe this happened... lol :D #fail 😅
4,user_5,Japan,Female,39,¡Esto es perfecto! 😊💃 #fiesta


In [3]:
df.shape

(100, 5)

In [7]:
df['language'] = df['tweet'].apply(lambda x: detect(x))
df.head()

,username,location,gender,age,tweet,language
0,user_1,India,Female,43,Can't believe this happened... lol :D #fail 😅,en
1,user_2,France,Male,26,信じられない… 😭💔,ja
2,user_3,Germany,Other,30,Das ist fantastisch 😍💯! #Liebe,de
3,user_4,Germany,Female,38,Can't believe this happened... lol :D #fail 😅,en
4,user_5,Japan,Female,39,¡Esto es perfecto! 😊💃 #fiesta,es


In [12]:
df_en = df[df['language'] == 'en'].reset_index(drop=True)
df_en.head()

,username,location,gender,age,tweet,language
0,user_1,India,Female,43,Can't believe this happened... lol :D #fail 😅,en
1,user_4,Germany,Female,38,Can't believe this happened... lol :D #fail 😅,en
2,user_21,France,Male,28,Can't believe this happened... lol :D #fail 😅,en
3,user_32,Japan,Female,48,I love this! 😍 Soooo good!!! #awesome 😊,en
4,user_43,Germany,Female,29,Can't believe this happened... lol :D #fail 😅,en


# NLP Preprocessing --- Convert to Lowercase

In [13]:
'NLP'.lower()

'nlp'

In [15]:
df_en['cleaned'] = df_en['tweet'].apply(lambda x: x.lower())
df_en.head()

,username,location,gender,age,tweet,language,cleaned
0,user_1,India,Female,43,Can't believe this happened... lol :D #fail 😅,en,can't believe this happened... lol :d #fail 😅
1,user_4,Germany,Female,38,Can't believe this happened... lol :D #fail 😅,en,can't believe this happened... lol :d #fail 😅
2,user_21,France,Male,28,Can't believe this happened... lol :D #fail 😅,en,can't believe this happened... lol :d #fail 😅
3,user_32,Japan,Female,48,I love this! 😍 Soooo good!!! #awesome 😊,en,i love this! 😍 soooo good!!! #awesome 😊
4,user_43,Germany,Female,29,Can't believe this happened... lol :D #fail 😅,en,can't believe this happened... lol :d #fail 😅


# NLP preprocessing --- Stopword Removal

In [20]:
stop_words  = set(stopwords.words('english'))

In [28]:
' '.join(word_tokenize("Can't believe this happened... lol :D #fail"))

"Ca n't believe this happened ... lol : D # fail"

In [24]:
df_en['cleaned'] = df_en['cleaned'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))
df_en.head()

,username,location,gender,age,tweet,language,cleaned
0,user_1,India,Female,43,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅
1,user_4,Germany,Female,38,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅
2,user_21,France,Male,28,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅
3,user_32,Japan,Female,48,I love this! 😍 Soooo good!!! #awesome 😊,en,love ! 😍 soooo good ! ! ! # awesome 😊
4,user_43,Germany,Female,29,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅


# NLP Preprocessing - BPE - Sub word Tokenization

In [30]:
# Step: 1 --> Prepare file for BPE Training

with open("bpe_train.txt", "w", encoding="utf-8") as f:
    for text in df_en['cleaned']:
        f.write(text + "\n")



In [31]:
# Step: 2 --> Initalize Tokenizer with BpeTrainer

tokenizer = Tokenizer(BPE())
trainer = BpeTrainer()
tokenizer.pre_tokenizer = Whitespace()

In [34]:
# Step: 3 --> Train tokenizer

tokenizer.train(files = ["bpe_train.txt"], trainer=trainer)

df_en['bpe_tokens'] = df_en['cleaned'].apply(lambda x: tokenizer.encode(x).tokens)
df_en.head()

,username,location,gender,age,tweet,language,cleaned,bpe_tokens
0,user_1,India,Female,43,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅,"[ca, n, ', t, believe, happened, ..., lol, :, ..."
1,user_4,Germany,Female,38,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅,"[ca, n, ', t, believe, happened, ..., lol, :, ..."
2,user_21,France,Male,28,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅,"[ca, n, ', t, believe, happened, ..., lol, :, ..."
3,user_32,Japan,Female,48,I love this! 😍 Soooo good!!! #awesome 😊,en,love ! 😍 soooo good ! ! ! # awesome 😊,"[love, !, 😍, soooo, good, !, !, !, #, awesome, 😊]"
4,user_43,Germany,Female,29,Can't believe this happened... lol :D #fail 😅,en,ca n't believe happened ... lol : # fail 😅,"[ca, n, ', t, believe, happened, ..., lol, :, ..."


In [37]:
list(df_en['bpe_tokens'])

[['ca',
  'n',
  "'",
  't',
  'believe',
  'happened',
  '...',
  'lol',
  ':',
  '#',
  'fail',
  '😅'],
 ['ca',
  'n',
  "'",
  't',
  'believe',
  'happened',
  '...',
  'lol',
  ':',
  '#',
  'fail',
  '😅'],
 ['ca',
  'n',
  "'",
  't',
  'believe',
  'happened',
  '...',
  'lol',
  ':',
  '#',
  'fail',
  '😅'],
 ['love', '!', '😍', 'soooo', 'good', '!', '!', '!', '#', 'awesome', '😊'],
 ['ca',
  'n',
  "'",
  't',
  'believe',
  'happened',
  '...',
  'lol',
  ':',
  '#',
  'fail',
  '😅'],
 ['love', '!', '😍', 'soooo', 'good', '!', '!', '!', '#', 'awesome', '😊'],
 ['love', '!', '😍', 'soooo', 'good', '!', '!', '!', '#', 'awesome', '😊'],
 ['ca',
  'n',
  "'",
  't',
  'believe',
  'happened',
  '...',
  'lol',
  ':',
  '#',
  'fail',
  '😅']]